In [1]:
!git clone https://github.com/Prashant-AV/Qualcomm-DL-Hackathon.git

Cloning into 'Qualcomm-DL-Hackathon'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 10 (delta 1), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (10/10), 30.68 MiB | 14.62 MiB/s, done.
Resolving deltas: 100% (1/1), done.


In [2]:
import pandas as pd

train_data = pd.read_csv('/content/Qualcomm-DL-Hackathon/train/train.csv')
train_data.head()
#train_data.shape

,image_names,emergency_or_not
0,1503.jpg,0
1,1420.jpg,0
2,1764.jpg,0
3,1356.jpg,0
4,1117.jpg,0


In [3]:
test_data = pd.read_csv('/content/Qualcomm-DL-Hackathon/test.csv')
test_data.head()
#train_data.shape

,image_names
0,1960.jpg
1,668.jpg
2,2082.jpg
3,808.jpg
4,1907.jpg


In [4]:
sample_data = pd.read_csv('/content/Qualcomm-DL-Hackathon/sample_submission.csv')
sample_data.head()
#train_data.shape

,image_names,emergency_or_not
0,1960.jpg,0
1,668.jpg,0
2,2082.jpg,0
3,808.jpg,0
4,1907.jpg,0


In [5]:
import zipfile
images_dir = '/content/Qualcomm-DL-Hackathon/train/images'
with zipfile.ZipFile('/content/Qualcomm-DL-Hackathon/train/images part-1.zip', 'r') as zip_ref:
    zip_ref.extractall(images_dir)

with zipfile.ZipFile('/content/Qualcomm-DL-Hackathon/train/images part-2.zip', 'r') as zip_ref:
    zip_ref.extractall(images_dir)

In [6]:
import shutil
import os

# Source and destination directories
source_dir1 = '/content/Qualcomm-DL-Hackathon/train/images/images part-1'
source_dir2 = '/content/Qualcomm-DL-Hackathon/train/images/images part-2'
destination_dir = '/content/Qualcomm-DL-Hackathon/dataset/images'
os.makedirs(destination_dir, exist_ok=True)

def copy_files(source, destination):
    for filename in os.listdir(source):
        source_file = os.path.join(source, filename)
        destination_file = os.path.join(destination, filename)
        if os.path.isfile(source_file):
          shutil.copy(source_file, destination_dir)


# Copy files from 'images part-1'
copy_files(source_dir1, destination_dir)

# Copy files from 'images part-2'
copy_files(source_dir2, destination_dir)

print("Files copied successfully!")

Files copied successfully!


In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader, random_split
from PIL import Image
import pandas as pd
import os
import torch.nn.functional as F


In [8]:
class EmergencyVehicleDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.data = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.data.iloc[idx, 0])
        image = Image.open(img_name)

        if self.data.shape[1] > 1:
            label = self.data.iloc[idx, 1]
        else:
            label = 0

        if self.transform:
            image = self.transform(image)
        return image, label


In [67]:
# Define transformations for the training and test datasets
train_transforms = transforms.Compose([
    transforms.RandomResizedCrop(224),  # Random crop and resize to 128x128
    transforms.RandomHorizontalFlip(),  # Randomly flip the image horizontally
    transforms.RandomRotation(20),      # Random rotation of up to 20 degrees
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),  # Random color jitter
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # ImageNet mean/std
])

test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    #transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
])


In [68]:

# Load the dataset
train_csv_dir = '/content/Qualcomm-DL-Hackathon/train/train.csv'
test_csv_dir = '/content/Qualcomm-DL-Hackathon/test.csv'
train_dataset = EmergencyVehicleDataset(csv_file=train_csv_dir, root_dir='/content/Qualcomm-DL-Hackathon/dataset/images', transform=transform)

# Split training data into training and validation sets
train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_data, val_data = random_split(train_dataset, [train_size, val_size])

# Create DataLoaders
batch_size = 32
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=True)
#test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [69]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [70]:
import torchvision.models as models

# GoogleNet model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.googlenet(pretrained=True)

# Modify the fully connected layer for your specific task
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)  # 2 output classes
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=GoogLeNet_Weights.IMAGENET1K_V1`. You can also use `weights=GoogLeNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [71]:
# Train the model
previous_loss = 0.0
num_epochs = 500
for epoch in range(num_epochs):  # Number of epochs
    model.train()
    running_loss = 0.0
    total = 0
    correct = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)

        # Reshape labels to have the same shape as outputs
        #labels = labels.unsqueeze(1).float()  # Add a dimension and convert to float
        # Apply sigmoid activation and squeeze to get probabilities

        # Convert labels to float
        #labels = labels.unsqueeze(1).float()

        loss = criterion(outputs, labels)
        loss.backward()
        #nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)  # Apply gradient clipping
        optimizer.step()
        running_loss += loss.item()

        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_loader)
    epoch_accuracy = 100 * correct / total
    if abs(epoch_loss ) < 0.2:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss : .4f}, Accuracy: {epoch_accuracy:.2f}%')
        print(f"Training stopped early at epoch {epoch+1} due to minimal loss change.")
        break

    previous_loss = epoch_loss
    if (epoch+1) % 1 == 0:
      print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss : .4f}, Accuracy: {epoch_accuracy:.2f}%')

print('Finished Training')

Epoch [1/500], Loss:  0.2652, Accuracy: 89.89%
Epoch [2/500], Loss:  0.1080, Accuracy: 96.58%
Training stopped early at epoch 2 due to minimal loss change.
Finished Training


In [72]:
# Evaluate the model on the validation set
model.eval()  # Set the model to evaluation mode
val_correct = 0
val_total = 0
with torch.no_grad():
    for inputs, labels in val_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        val_total += labels.size(0)
        val_correct += (predicted == labels).sum().item()

val_accuracy = 100 * val_correct / val_total
print(f'Validation Accuracy: {val_accuracy:.2f}%')

Validation Accuracy: 94.55%


In [73]:
test_dataset = EmergencyVehicleDataset(csv_file=test_csv_dir, root_dir='/content/Qualcomm-DL-Hackathon/dataset/images', transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Make predictions on the test data
predictions = []
model.eval()  # Set the model to evaluation mode
with torch.no_grad():
    for images, _ in test_loader:
        images = images.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        predictions.extend(predicted.cpu().numpy())

# Create a submission dataframe
submission_df = pd.DataFrame({'image_names': test_data['image_names'], 'emergency_or_not': predictions})
submission_df.to_csv('submission.csv', index=False)

print("Predictions saved to submission.csv")

Predictions saved to submission.csv
